In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [6]:
!git clone https://github.com/Vela-Yang/vela_SR.git SR

Cloning into 'SR'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 4), reused 13 (delta 4), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch

from SR.models import AVS3Filter
from SR.dataset import MyDataset
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
from tqdm.notebook import tqdm


# 是否有GPU加速
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False



In [ ]:
# 加载网络模型
model = AVS3Filter().to(device)

# 损失函数
loss_fun = torch.nn.MSELoss().to(device)
losses = []
losses_test = []

# 数据集加载
train_set = VelaDataset(root="./drive/MyDrive/iaprtc12/", number_of_files=19140, subset="training")
test_set = VelaDataset(root="./drive/MyDrive/iaprtc12/", number_of_files=19140, subset="testing")

In [ ]:
# 优化器
optimizer = torch.optim.Adam(model.parameters())
# 批大小
batch_size = 2

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory
)
test_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=pin_memory
)
loss_fun = torch.nn.MSELoss()

loss_fun = torch.nn.MSELoss().to(device)
losses = []
losses_test = []


In [ ]:
# 训练函数
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, images in enumerate(train_loader):
        y, y_lr = images
        output = model(y_lr.to(device))
        loss = loss_fun(output, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 展示训练状态
        if batch_idx % log_interval == 0:
            print(
                f"迭代次数: {epoch} [{batch_idx * len(y_lr)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\t训练损失: {loss.item():.6f}")
        # 更新状态栏
        pbar.update(pbar_update)
        # 记录训练损失
        losses.append(loss.item())


# 测试
def test(model, epoch):
    model.eval()
    for y, y_lr in test_loader:
        y_lr = y_lr.to(device)
        y = y.to(device)
        output = model(y_lr)
        loss = loss_fun(output, y)
        losses_test.append(loss)
        # 更新状态栏
        pbar.update(pbar_update)
    print(f"\n迭代次数 {epoch}\t训练损失: {loss.item():.6f}")


In [8]:

n_epoch = 100
log_interval = 1
pbar_update = 1 / (len(train_loader) + len(test_loader))

with tqdm(total=n_epoch) as pbar:   
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)

# 展示训练损失与准确率
plt.subplot(2, 1, 1)
plt.plot(losses);
plt.title("训练损失");
plt.subplot(2, 1, 2)
plt.plot(losses_test);
plt.title("测试损失");



RuntimeError: ignored